In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hwk03_personal_networks.ipynb")

In [ ]:
!pip install --upgrade networkx
!pip install --upgrade pandoc

In [ ]:
from IPython.core.display import HTML
from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# Homework 3 

### Personal network surveys

In this assignment, we will analyze some data from the [General Social Survey](http://gss.norc.org/) (GSS).
The GSS is the survey that was the basis of the debate over whether or not Americans are becoming more socially isolated, which we discussed in class.

In [ ]:
import os
os.getcwd()

The file `GSS.csv` has an extract from the GSS which we will analyze today.

Here is the [codebook](http://gss.norc.org/documents/codebook/GSS_Codebook.pdf) for the entire GSS. Of course, the GSS is huge, so you will have to search through for the variable names included in this extract. (Don't do this by hand -- use your pdf viewer's search function.)

Read the GSS extract into a Table called `gss_data`.

In [ ]:
gss_data = Table.read_table('GSS.csv')

## Question 1:
How many rows and how many columns does `gss_data` have? Print out the first several rows to take a look at its contents.

In [ ]:
gss_number_of_rows = ...
gss_number_of_cols = ...

print("num rows: ", gss_number_of_rows)
print("num cols: ", gss_number_of_cols)

In [ ]:
grader.check("q1")

In [ ]:
gss_data

## Question 2:
What range of years is covered by this dataset? Answer this by finding the largest and smallest year. 

_Hint: You can use the functions max and min that we introduced back in Lab 0_

In [ ]:
largest_year = ...
smallest_year = ...

print("earliest year: ", smallest_year)
print("latest year: ", largest_year)

In [ ]:
grader.check("q2")

We are interested in the years when the 'important matters' question was asked of survey respondents. It turns out that this question was only asked for 1985 and 2004. In order to continue with our analysis, we will pick out only the rows of the dataset that correspond to the years we are interested in.

## Question 3:
Make two new datasets: `gss_1985` and `gss_2004` which have only the responses from 1985 and from 2004.

In [ ]:
gss_1985 = ...
gss_2004 = ...

In [ ]:
grader.check("q3")

## Question 4: 
How many responses are there from 1985, and how many from 2004? 

In [ ]:
responses_from_1985 = ...
responses_from_2004 = ...

In [ ]:
grader.check("q4")

## Question 5:
Make a table of the responses to the `numgiven` question for each year. This tables should show the possible options for `numgiven` and how often that response occurred.

In [ ]:
gss_1985_numgiven = ...
gss_1985_numgiven

In [ ]:
gss_2004_numgiven = ...
gss_2004_numgiven

In [ ]:
grader.check("q5")

Your table for 2004 should show that quite a few respondents have -1 as the value of `numgiven`. These respondents actually did not answer the important matters name generator.

## Question 6:
Narrow the 2004 dataset down so that it does not have the respondents who have -1 values for `numgiven`

_Hint: If needed, look back at Lab2 for examples of using `where` and `are.equal_to` and modify to `are.above`_

In [ ]:
gss_2004_interviewed = ...

In [ ]:
grader.check("q6")

## Question 7:
Narrow both datasets down so that they only have respondents who were asked the `numgiven` question and who provided answers to that question.

In [ ]:
gss_1985.where('numgiven', are.not_equal_to(9)).where('numgiven', are.above(-1))

In [ ]:
gss_1985.where('numgiven', are.not_equal_to(9))
# gss_2004_interviewed.where('numgiven', are.not_equal_to(9))

In [ ]:
gss_1985_responded = ...
gss_2004_responded = ...

In [ ]:
grader.check("q7")

Now we have the set of respondents we will study in more detail: those who responded to the 'important matters' name generator.

**For the problems below, please use `gss_1985_responded` and `gss_2004_responded`.**

Many analysts have focused on how many survey respondents report that they don't discuss important matters with anyone.  They interpret the fraction of respondents who don't report discussing important matters with anyone as an indicator for the amount of social isolation. (These respondents who report not discussing important matters with anyone have `numgiven` equal to 0.)

<!-- BEGIN QUESTION -->

## Question 8:
Do you think this is a good way to try to quantify social isolation? Name one way this could be a good measure of social isolation, and one way this could be a bad measure of social isolation. Please be specific.

_Type your answer here, replacing this text._

<!-- END QUESTION -->

## Question 9:
For both the 1985 and 2004 datasets, create a new variable, `isolated` which has the value False if the respondent reports discussing important matters with anyone, and True otherwise.


In [ ]:
isolated_1985 = ...
isolated_2004 = ...

In [ ]:
grader.check("q9")

## Question 10:
Using the variable you just created, what proportion of respondents was socially isolated in 1985? In 2004?

In [ ]:
proportion_isolated_1985 = ...
proportion_isolated_2004 = ...

print("Proportion isolated in 1985: ", proportion_isolated_1985)
print("Proportion isolated in 2004: ", proportion_isolated_2004)

In [ ]:
grader.check("q10")

In [ ]:
gss_1985

Here is a function that you may find useful in answering the next question. Given a row in a GSS dataset, the function returns `True` if one of the alters is a spouse, and `False` otherwise:

In [ ]:
def reports_spouse(row):
    return(row.item('spouse1') == 1 or row.item('spouse2') == 1 or row.item('spouse3') == 1 or row.item('spouse4') == 1 or row.item('spouse5') == 1)

## Question 11:
What proportion of married respondents named a spouse?

In [ ]:
married_1985 = gss_1985_responded.where("marital", are.equal_to(...))
married_spouses_1985 = married_1985.apply(...)
married_spouses_proportion_1985 = ...

married_2004 = gss_2004_responded.where("marital", are.equal_to(...))
married_spouses_2004 = married_2004.apply(...)
married_spouses_proportion_2004 = ...
print("proportion of married respondents naming spouse in 1985: ", married_spouses_proportion_1985)
print("proportion of married respondents naming spouse in 2004: ", married_spouses_proportion_2004)

In [ ]:
grader.check("q11")

### Homophily

 Below, you will find the functions that we used to convert data from wide to long as part of Lab 2. The `wide_to_long` function has been slightly modified to account for the different format of the variable names in the GSS dataset, but it works in the same way we saw in the lab.

In [ ]:
def repeat_single_col(data, var_name, times=5):
    """Repeats a single column multiple times.
    
    Parameters
    ----------
    var_name : str
        Text that contains the name of the column to repeat.
    
    Returns
    -------
    np.array
        A single array with the contents of the column repeated five times.
    
    Examples
    --------
    >>> repeat_single_col(Table().with_columns(['respondent_age', [10]]),
                          'respondent_age')
    
    array([10, 10, 10, 10, 10])
    """
    new_col = np.tile(data.column(var_name), times)
    return new_col

def wide_to_long(data, var_name, times=5):
    """Given columns of alter characteristics, stack them into one long column.
    
    Parameters
    ----------
    data : Table
        The data table containing the alter characteristics
    var_name : str
        Text that contains the variable name; columns of the dataset should
        match the pattern: [var_name][alter_number]
        For example, if var_name is 'age' then this function expects to find
        columns in the survey dataset named 
        'age1', 'age2', 'age3', 'age4', and 'age5'
    times : int
        The number of columns for each characteristic
    
    Returns
    -------
    np.array
        A single array with the contents of all of the columns stacked on top of one another.
    
    Examples
    --------
    >>> wide_to_long(Table().with_columns(['age1', [10, 15],
                                           'age2', [30, 35],
                                           'age3', [20, 15],
                                           'age4', [60, 70],
                                           'age5', [20, 25]]),
                     'age')
    
    array([10, 15, 30, 35, 20, 15, 60, 70, 20, 25])
    """
    new_col = np.concatenate([data.column(var_name + str(idx)) for idx in range(1,times+1)])
    return new_col

## Question 12: 
Now we will use these functions to convert the wide-format data from 1985 and 2004 into long format. This will enable us to examine whether or not there is evidence of homophily in the GSS confidant reports from those two years.

Follow the pattern that we used in Lab 2 to do this. Be sure to include the following:

* respondent's age
* alter's gender (called 'sex' in the dataset')
* alter's age


In [ ]:
gss_1985_long_raw = ...
gss_2004_long_raw = ...

In [ ]:
grader.check("q12")

In [ ]:
gss_1985_responded.show(10)

## Question 13:
Not all respondents reported 5 alters. In cases where alter information is missing, `alter_age` is coded as -1. Furthermore, in cases where respondents did report about an alter, but they did not know or refused to give the alter's age, [the codebook](http://gss.norc.org/documents/codebook/GSS_Codebook.pdf) tells use that `alter_age` will have the value 98 or 99.

Create the Tables gss_1985_long and gss_2004_long, which start from `gss_1985_long_raw` and `gss_2004_long_raw` and filter out rows where `alter_age` equals -1, 98, or 99 so that we are left with only actual reported alters whose age was given.

Hint: use `.where()` to perform filtering.

In [ ]:
gss_1985_long = gss_1985_long_raw.where(...)
gss_1985_long = gss_1985_long.where(...)
gss_1985_long = gss_1985_long.where(...)

gss_2004_long = gss_2004_long_raw.where(...)
gss_2004_long = gss_2004_long.where(...)
gss_2004_long = gss_2004_long.where(...)

In [ ]:
grader.check("q13")

<!-- BEGIN QUESTION -->

## Question 14:
Create a scatterplot of the respondent's age and the alter's age (make a separate plot for 1985 and for 2004).

In [ ]:
gss_1985_long.scatter(...)

In [ ]:
gss_2004_long.scatter(...)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Question 15:
What do the scatter plots you made suggest about homophily in Americans' confidant networks? How similar or different are these patterns to what we saw from our survey of Berkeley students? (Note: Look back at Lab2. There is no single right answer here -- just interpret the plots and provide an argument for why your interpretation might be right.)

_Type your answer here, replacing this text._

<!-- END QUESTION -->

# Adjacency Matrices and Edge Lists

<!-- BEGIN QUESTION -->

## Question 16:
Consider the undirected graph as shown in the figure below.

<img src="fig_Graph1.png" height=40% width=40%>

1. Write down the adjacency matrix for this graph
2. Write down the edge list for the graph
3. Which representation is better for this graph and why?

(Your answers can and should be in text, not code; you can use triple backticks (like \`\`\` text here \`\`\`) to format text)

_Type your answer here, replacing this text._

<!-- END QUESTION -->

# Breadth first search

In class, we covered an example using the ARPANET graph that systematically compute the distance between one node (MIT) and every other node (lecture 2). The searching algorithm we used is called breadth first search. Use this algorithm and calculate the longest distance between two nodes in the following graph.

<img src="fig_bfs.png" width="240" height="180" align="center"/>
<br>

## Question 17:
Write down the steps in the following cell and enter the longest distance between each pair of nodes as q17.

_Hint: Array d1_a should include nodes that are directly connected to A. Array d2_a should include all nodes that are distance 2 from A. Array d3_a should include all nodes distance 3 from A. And so on..._

In [ ]:
# Begin with A
d1_a = make_array(...) # Nodes that have distance 1 with A
...
# Move on to all other nodes

q17 = ...

In [ ]:
grader.check("q17")

# SUBMIT YOUR ASSIGNMENT

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Please upload the .zip file to Gradescope.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)